# React useCallback 

## Introduction
- Topic: useCallback hook in React — what it is, why it exists, and practical use-cases. 
- Goal: Learn when to memoize a function reference, prevent unnecessary child re-renders, and optimize expensive operations by controlling function recreation. 

## What is useCallback?
- Definition: useCallback returns a memoized function reference so the function is not recreated on every render unless specified dependencies change. 
- Key distinction: useMemo memoizes a computed *value/result*, while useCallback memoizes a *function reference*. 

## Why functions get recreated
- On every re-render, functions declared inside a component are re-created, producing a new reference each time. 
- When a function reference passed as prop changes, child components see it as a changed prop and may re-render even if nothing else changed. 

## Primary use-cases (two main categories)
1. Prevent unnecessary child re-renders (with React.memo)  
   - Problem: A parent re-render causes functions to be recreated; passing those functions to children as props makes React.memo ineffective because the prop reference changes. 
   - Solution: Wrap the function in useCallback so the reference stays stable across renders unless dependencies change; combine with React.memo on child for actual render skipping. 
2. Control recreation for expensive operations  
   - Problem: Expensive functions (big calculations) recreated frequently can hurt performance in large apps. 
   - Solution: Wrap the expensive-operation function in useCallback and provide an appropriate dependency list so the function recreates only when necessary. This prevents unnecessary work related to function recreation (and can help avoid cascading child updates). 

## Syntax and behavior
- Basic form: const memoizedFn = useCallback(() => { /* function body */ }, [deps]); 
- Effects:
  - The function body still executes when explicitly called; useCallback only stabilizes the function reference, it does not prevent function execution. 
  - If dependency array is empty, the function reference is frozen to its initial closure values — may cause stale-closure problems unless intended. 

## Example 1 — Preventing child re-renders (counter + child button)
- Scenario: Parent has a count state and passes:
  - a button label prop (primitive) and
  - a handler function to child. 
- Without useCallback:
  - Parent re-render (count changes) recreates handler function; child receives a new function prop and re-renders even if label unchanged. React.memo cannot prevent this because prop reference changed. 
- With useCallback:
  - Wrap handler in useCallback with proper dependencies so the function reference remains the same across parent renders when dependencies do not change, preventing unnecessary child re-render. 

Important notes from the demo:
- If useCallback is used with an empty dependency array but the function reads state (e.g., count), it will "freeze" the state value in the closure and produce stale results (counter stops updating). This is because the memoized function captured the initial state value. 
- Fix: include relevant state (e.g., count) in the dependency array so the function is re-created when that state changes and it has fresh values. 

## Example 2 — Handling expensive operations
- Scenario: Component has:
  - count state (incremented via button)
  - text input state (typing updates text)
  - an "expensiveCalculation" function that loops over a large range or does heavy computation and returns a result. 
- Problem: Every render recreates expensiveCalculation; recreating the function reference may be costly or cause child effects. 
- Approach with useCallback:
  - Wrap expensiveCalculation in useCallback and include only the dependencies that should trigger its recreation (for example, include count if the calculation depends on count). 
  - This ensures the function reference is stable when irrelevant state (like text input) changes, avoiding unnecessary recreation. 
- Verification: Use a useRef and useEffect to compare previous and current function references and log when recreation happens (i.e., "Function got recreated"). This demonstrates that with correct dependencies the function only re-creates when expected. 

## Practical tips & gotchas
- useCallback does not prevent a function from running; it only stabilizes the reference. Running the function still performs work every time it is called. 
- Avoid empty dependency arrays when the function uses state or props that should update — that causes stale closures and incorrect behavior. 
- Prefer specifying only necessary dependencies: include values the function uses that must cause it to refresh. 
- Combine useCallback with React.memo on child components to effectively prevent unnecessary child re-renders. 
- For memoizing results (not function references) use useMemo instead. 
- Performance gains are most visible in large/complex DOM trees or expensive operations — small examples may not show measurable differences but practice is good for scaling apps. 

## Code patterns (concise)
- Stabilize handler:
  - const handleClick = useCallback(() => setCount(c => c + 1), [setCount]); — or include count as dependency if referencing its value directly. 
- Avoid stale closure:
  - Wrong: const handleClick = useCallback(() => setCount(count + 1), []); // stale count 
  - Right: const handleClick = useCallback(() => setCount(prev => prev + 1), []); // safe when using functional updater, or include count in deps 
- Expensive calc:
  - const expensiveCalc = useCallback(() => { /* heavy work using count */ }, [count]); 

## Small checklist for useCallback usage
- Is a function passed to a memoized child (React.memo)? If yes, consider useCallback to stabilize the prop. 
- Does the function perform or trigger expensive work on recreation? If yes, consider useCallback with correct deps. 
- Does the function read state or props that must be up-to-date? If yes:
  - include those values in dependency array, or
  - use functional updates (setState(prev => ...)) to avoid listing state as dep when appropriate. 
- Test for stale closure bugs: if behavior seems frozen after wrapping in useCallback, inspect dependency array and closure usage. 

## Examples explained
- Example: Counter + ChildButton
  - Parent: has count, handleClick; ChildButton receives label and handleClick. Without useCallback, ChildButton re-renders on every parent render; with useCallback and React.memo, ChildButton only re-renders when label or handleClick reference actually changes. 
- Example: Expensive calculation component
  - expensiveCalculation runs over a large range and returns result; wrapping it in useCallback with count in deps ensures recreation only when count changes, not on unrelated input updates. Use useRef + useEffect to log function recreation and verify behavior. 

## Important terms
- Memoization: caching a value or reference to avoid recomputation or recreation. 
- Stale closure: a memoized function capturing outdated state/props because dependencies were not included. 
- React.memo: a HOC that prevents re-rendering a functional child component when props are shallow-equal. It compares prop references. 

## Final takeaways
- useCallback memoizes function references; use useCallback when passing functions to memoized children or when avoiding unnecessary function re-creation for performance reasons. 
- Always supply correct dependencies; empty arrays can freeze closures and cause bugs. 
- useCallback helps at scale — apply it where component trees or expensive operations justify the added complexity. 
